In [4]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import EarlyStopping
from keras import initializers
from math import sqrt
df = yf.download("BTC-USD", period="5y", interval="1d")
import warnings
warnings.filterwarnings("ignore")

[*********************100%%**********************]  1 of 1 completed


In [5]:
import numpy as np
import yfinance as yf
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import pickle

# Assuming the scaler and model are already loaded
scaler = pickle.load(open('scaler.pkl', 'rb'))
model = load_model('lstm_model.h5')

# Download the latest data
df_new = yf.download("BTC-USD", period="3mo", interval="1d")
df_new.dropna(inplace=True)

# Assuming 'look_back' was used during model training
look_back = 7  # Ensure this matches the number of features the model was trained with

# Scale the last 'look_back' closing prices to match the model's training conditions
last_values = df_new['Close'].values[-look_back:]
last_values_scaled = scaler.transform(last_values.reshape(-1, 1))

# Correct reshaping to fit the model's expected input: (samples, time steps, features)
last_values_scaled = last_values_scaled.reshape(1, 1, look_back)

# Predict using the correctly shaped data
predicted_price = model.predict(last_values_scaled)
predicted_price_unscaled = scaler.inverse_transform(predicted_price)  # If you want the price in original scale

print(f"Predicted next price: {predicted_price_unscaled[0][0]}")


[*********************100%%**********************]  1 of 1 completed
1/1 [==============================] - 0s 289ms/step
Predicted next price: 61506.9296875


In [6]:
import plotly.graph_objs as go
import plotly.offline as py
from datetime import timedelta
# Prepare the plot
latest_date = df_new.index.max()
next_date = latest_date + timedelta(days=1)  # assuming daily data, next day prediction

# Historical data trace
trace1 = go.Scatter(
    x = df_new.index,
    y = df_new['Close'],
    mode = 'lines',
    name = 'Historical Prices',
    line = dict(color=('rgb(66, 134, 244)'), width=2)
)

# Predicted price trace
trace2 = go.Scatter(
    x = [next_date],
    y = [predicted_price_unscaled[0][0]],
    mode = 'markers',
    name = 'Predicted Price',
    marker = dict(color=('rgb(244, 146, 65)'), size=10)
)

data = [trace1, trace2]
layout = go.Layout(
    title = 'Bitcoin Price Forecast',
    xaxis = dict(title = 'Date'),
    yaxis = dict(title = 'Price, USD')
)
fig = go.Figure(data=data, layout=layout)

# Display the plot
py.iplot(fig, filename='bitcoin_price_forecast')